# LISTA 1

## GLOBAL SETUP

In [38]:
import numpy as np
import os
from tqdm import tqdm
import nltk
import collections
import random

## ZADANIE 1

In [27]:
if not os.path.isfile("sjp-20201007.zip"):
    print("doing a succ on SJP dictionary")
    os.system("wget https://sjp.pl/slownik/growy/sjp-20201007.zip")
else:
    print("SJP already downloaded, unpacking")

if not os.path.isfile("slowa.txt"):
    os.system("unzip sjp-20201007.zip")

words = []
with open("slowa.txt", "r") as f:
    words = set(f.read().split("\n"))
    
# the dict is mentally challenged and does not have 'i' in it
words.add("i")
words.add("z")


longest = max(len(w) for w in words)    

print(f"read {len(words)} words from SJP, longest one is {longest} letters long!")

SJP already downloaded, unpacking
read 3045880 words from SJP, longest one is 15 letters long!


In [28]:
def MaxMatch(w, verbose = False):
    tokenized = []
    succ = 1
    best = (0, "reeee")
    
    while len(w) > 0:
        try_ = w[:succ]
        if try_ in words:
            if verbose:
                print(f"found candidate {try_}")
            best = (succ, try_) 

        succ += 1
        
        if succ > longest or try_ == w:
            if best[0] == 0:
                raise Exception(f"could not match {w[:succ]}")
            else:
                if verbose:
                    print(f"longest match was {best[1]}")
                tokenized.append(best[1])
                if verbose:
                    print(f"transforming {w} -> {w[best[0]:]}")
                w = w[best[0]:]
                succ = 1
                best = (0, "reeee")
    return tokenized
                
MaxMatch("kamienietestowanie")

['kamienie', 'testowanie']

## ZADANIE 2 i 3


In [25]:
def calculate_bigrams():
    text = []
    for dir in tqdm(os.listdir("corpora/")):
        with open(f"corpora/{dir}/text.xml", "r") as corpfile:
            corpseg = corpfile.read().split("\n")
            for l in corpseg:
                if "<ab" in l:
                    l = l.replace("</ab>", "")
                    l = l.split(">")[1]
                    text.append(l.lower())
    print(f"got {len(text)} snippets")
    
    bigram_stat = collections.defaultdict(lambda: 0)
    for sentence in tqdm(text):
        tokens = nltk.word_tokenize(sentence)
        for k in range(len(tokens) - 1):
            key = f"{tokens[k]}#{tokens[k+1]}"
            bigram_stat[key] += 1
            
    baked_bigrams = [(bigram_stat[k], k) for k in bigram_stat.keys()]
    baked_bigrams = sorted(baked_bigrams, reverse=True)
    return baked_bigrams
    
bigrams = calculate_bigrams()

  1%|          | 418/39566 [00:00<00:09, 4169.32it/s]

got 39566 snippets


100%|██████████| 662051/662051 [00:00<00:00, 2255949.04it/s]


0

In [36]:
bigram_dict = collections.defaultdict(lambda: [])
for bfreq, bigram in tqdm(bigrams):
    if bigram.count("#") > 1:
        continue
    primary, secondary = bigram.split("#")
    bigram_dict[primary].append((bfreq, secondary))

100%|██████████| 662051/662051 [00:01<00:00, 415527.07it/s]


In [71]:
def generate_story(max_len, starting, distribution):
    if max_len == 0:
        max_len = 1000000
    if starting == None:
        starting = random.choice(list(bigram_dict.keys()))
        
    while(max_len > 0):
        print(starting, end = ' ')
        max_len -= 1
        candidates = bigram_dict[starting]
        
        if distribution == "random":
            candidates = [c[1] for c in candidates]
            if len(candidates) == 0:
                return
            starting = random.choice(candidates) 
            
        if distribution == "weight":
            if len(candidates) == 0:
                return
            all_ents = sum(c[0] for c in candidates)
            chosen = random.randint(0, all_ents)
            for it in range(len(candidates)):
                chosen -= candidates[it][0]
                if chosen <= 0:
                    starting = candidates[it][1]
                    break
                    
            

In [72]:
for i in range(10):
    generate_story(20, "papież", "weight")
    print("\n=======")

papież cierpi z zastrzeżeniem ust . kierunki , gdzie z polską walutę wymienialną ( ironia ... a prawko to niezły 
papież zostałby w nocy , można było żyć bez pracy , ale to nie mogliśmy podać niemało trzy godziny wieczorne 
papież nawiązał do takiego czy na polskim '' . im pod mantyneją . dla moich kapłanów w nowym domu wiesława 
papież się w formie komunikatu wydanego w okolicy było zgodnie z tym , mających zastosowanie przepisy prawne rozróżnienie wydaje ten 
papież benedykt xvi wieku , z powodu pewnej chwili do suwałk . wszystko jedno zdanie z otwartymi oczami . nie 
papież jan sibiga , po 1 . cóż dopiero przed redakcją na kempingu samochodowego secz we wściekłość holendra spotęgowały rzęsiste 
papież , dawniej tłumacz i realizować . nie chcąc nie posiadała znaczne zbliżenie . drzwi , jeden plus procent to 
papież zwołał też lubił kobiety są jednak wycieku testów przydatności w rękę , helu . nie chce nie przeszkadzała w 
papież nawiązał kontakt z dyrektywami przerwały po zestawieniu